In [32]:
import pandas as pd
import plotly.express as px
import ipywidgets as ipw
from IPython.display import display


#  vs code muestra los gráficos de plotly de forma nativa
#  Jupyter Notebook del explorador no muestra los gráficos de plotly de forma nativa
# Descomentar lo siguiente si se quiere usar Plotly en Jupyter Notebook del explorador

import plotly.io as pio

In [ ]:
## cargado de tablas hechas por AnalisisOfertas.ipynb
chile = pd.read_csv("regiones-chile.csv", sep=";",encoding="utf-8")
tabla_puestos_comunas_total = pd.read_csv("../data/tabla_puestos_comunas_total.csv", sep=",", encoding="utf-8")
promedio_salario_por_puesto = pd.read_csv("../data/promedio_salarios_por_puesto.csv", sep=",", encoding="utf-8")
promedio_salarios_total = pd.read_csv("../data/promedio_salarios_por_ciudad_comuna.csv", sep=",", encoding="utf-8")
tabla_ciudades_total = pd.read_csv("../data/total_ofertas_por_ciudad.csv", sep=",", encoding="utf-8")


# Rehacer gráfico de barras de ofertas por región por error en la anterior iteracion
# Crear un diccionario para mapear comuna a región
comuna_a_region = chile.set_index('Comuna')['Región'].to_dict()

# Clasificar cada ciudad/comuna y sumar la cantidad de ofertas por región
tabla_ciudades_total['Región'] = tabla_ciudades_total['Ciudad/Comuna'].map(lambda x: comuna_a_region.get(x.lower(), comuna_a_region.get(x.capitalize(), None)))

# Agrupar por región y sumar las ofertas
ofertas_por_region = (
    tabla_ciudades_total.groupby('Región', as_index=False)['Cantidad_Ofertas']
    .sum()
    .sort_values('Región')
    .reset_index(drop=True)
)


In [44]:
def grafico_iteract_ofertas_por_ciudad(comuna, plot_output=None):  
    # Filtrar los datos para la comuna seleccionada
    datos = tabla_puestos_comunas_total[tabla_puestos_comunas_total['Comuna'].str.lower() == comuna.lower()]
    if datos.empty:
        print(f"No hay datos para la comuna '{comuna}'.")
        return
    fig = px.pie(
        datos,
        names="Puesto",
        values="Cantidad_Ofertas",
        title=f"Distribución de Puestos en {comuna.capitalize()}",
        hole=0.4,
        custom_data=["Puesto", "Cantidad_Ofertas"]
    )
    fig.update_traces(
        textinfo='label',
        hovertemplate='<b>Puesto:</b> %{label}<br><b>Cantidad de ofertas:</b> %{value}<extra></extra>'
    )
    fig.update_layout(
        transition=dict(
            duration=500,  # milisegundos
            easing="cubic-in-out"
        )
    )

    if plot_output is not None:
        with plot_output:
            plot_output.clear_output(wait=True)
            fig.show()
    else:
        fig.show()

In [45]:
def grafico_ofertas_por_region_interactivo(ofertas_por_region=ofertas_por_region, plot_output=None):
    fig = px.bar(
        ofertas_por_region,
        x="Región",
        y="Cantidad_Ofertas",
        labels={'Región': 'Región', 'Cantidad_Ofertas': 'Cantidad de Ofertas'},
        title='Cantidad de Ofertas Laborales por Región',
        color_discrete_sequence=['skyblue']
    )
    fig.update_layout(
        xaxis_title='Región',
        yaxis_title='Cantidad de Ofertas',
        transition=dict(
            duration=500,
            easing="cubic-in-out"
        )
    )
    fig.update_traces(
        hovertemplate='<b>Región:</b> %{x}<br><b>Cantidad de ofertas:</b> %{y}<extra></extra>'
    )
    fig.update_yaxes(type='log')
    if plot_output is not None:
        with plot_output:
            plot_output.clear_output(wait=True)
            fig.show()
    else:
        fig.show()


In [ ]:

region_dropdown = ipw.Dropdown(description="Región:", layout=ipw.Layout(width="300px"))
provincia_dropdown = ipw.Dropdown(description="Provincia:", layout=ipw.Layout(width="300px"))
comuna_dropdown = ipw.Dropdown(description="Comuna:", layout=ipw.Layout(width="300px"))

rango_dropdown = ipw.IntSlider(
    value=0,
    min=0,
    max=200000000,
    step=1,
    description='Salario:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

puesto_dropdown = ipw.Dropdown(
    options=["Ninguno", "Frontend", "Backend", "Data Analyst"],
    description='Puesto:',
    layout=ipw.Layout(width='250px')
)

# -------- KPIs --------
kpi_empleos = ipw.HTML(value="<h3 style='text-align:center;'>100<br><span style='font-size:14px'>Cantidad total de empleos</span></h3>")
kpi_salario = ipw.HTML(value="<h3 style='text-align:center;'>$1.200.000<br><span style='font-size:14px'>Salario Promedio</span></h3>")

kpi_box_1 = ipw.HBox([ipw.Label("🔢"), kpi_empleos])
kpi_box_2 = ipw.HBox([ipw.Label("💰"), kpi_salario])
kpi_section = ipw.HBox([kpi_box_1, kpi_box_2])

# -------- Imagen o gráfico placeholder --------
plot_output = ipw.Output()
with plot_output:
    plot_output.clear_output(wait=True)
    grafico_ofertas_por_region_interactivo(plot_output=plot_output)



def actualizar_kpis():
    # Si hay una comuna seleccionada, filtrar por comuna
    comuna = comuna_dropdown.value
    if comuna:
        # Filtrar tabla_ciudades_total por la comuna seleccionada
        filtro = tabla_ciudades_total["Ciudad/Comuna"].str.lower() == comuna.lower()
        total = tabla_ciudades_total.loc[filtro, "Cantidad_Ofertas"].sum()  
        # filtrar promedio_salarios_total por la comuna seleccionada
        filtro_salario = promedio_salarios_total["Ciudad/Comuna"].str.lower() == comuna.lower() # type: ignore
        # Calcular el promedio de salario solo para la comuna seleccionada
        promedio = promedio_salarios_total.loc[filtro_salario, "Promedio_Salario"].sum() # type: ignore
    else:
        total = tabla_ciudades_total["Cantidad_Ofertas"].sum()
        promedio = promedio_salarios_total["Promedio_Salario"].mean()  # type: ignore

    kpi_empleos.value = f"<h3 style='text-align:center;'>{int(total)}<br><span style='font-size:14px'>Cantidad total de empleos</span></h3>"
    kpi_salario.value = f"<h3 style='text-align:center;'>${promedio:,.0f}<br><span style='font-size:14px'>Salario Promedio</span></h3>"
    # -------- Layout general por pestaña --------   
    return None


##
def actualizar_provincias(change):        
    region = change['new']
    provincias = chile[chile["Región"] == region]["Provincia"].unique()
    provincia_dropdown.options = sorted(provincias)
    
    # Resetear comunas
    comuna_dropdown.options = []

def actualizar_comunas(change):
    provincia = change['new']
    comunas = chile[chile["Provincia"] == provincia]["Comuna"].unique()
    comuna_dropdown.options = sorted(comunas)


def tab_1():
    filtros = ipw.VBox([region_dropdown, provincia_dropdown, comuna_dropdown, rango_dropdown])
    derecha = ipw.VBox([
            plot_output,
    ], layout=ipw.Layout(width='600px', min_width='400px', max_width='800px', align_items='flex-start'))
    layout = ipw.HBox([
        filtros,
        ipw.VBox([kpi_section, derecha], layout=ipw.Layout(margin='0 0 0 40px'))
    ])
    return layout

##
region_dropdown.options = sorted(chile["Región"].unique())
provincia_dropdown.options = []
comuna_dropdown.options = []
actualizar_kpis()  # Inicializar KPIs al cargar la pestaña

region_dropdown.observe(actualizar_provincias, names='value')
provincia_dropdown.observe(actualizar_comunas, names='value')
def on_comuna_change(change):
    comuna = change['new']
    actualizar_kpis()  # Actualizar KPIs al cambiar la comuna
    if comuna:
        with plot_output:
            plot_output.clear_output(wait=True)
            grafico_iteract_ofertas_por_ciudad(comuna, plot_output=plot_output)
    else:
        with plot_output:
            plot_output.clear_output(wait=True)
            grafico_ofertas_por_region_interactivo(plot_output=plot_output)
        

comuna_dropdown.observe(on_comuna_change, names='value')


# Otras pestañas vacías por ahora
tab_2 = ipw.HTML("<h3>Tecnologías (en desarrollo)</h3>")
tab_3 = ipw.HTML("<h3>Seguridad (en desarrollo)</h3>")

# -------- Tabs dashboard --------
tabs = ipw.Tab()
tabs.children = [tab_1(), tab_2, tab_3]
tabs.set_title(0, "Cantidad de empleos y salarios")
tabs.set_title(1, "Tecnologías")
tabs.set_title(2, "Seguridad")

display(tabs)